In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def create_schedule_matrix():
    """Crea una matriz de horario vacía (5 días x 9 bloques)."""
    return [[False for _ in range(9)] for _ in range(5)]

def get_day_index(day):
    """Convierte el nombre del día a índice."""
    if not isinstance(day, str):
        print(f"Error: Día inválido: {day}")
        return -1
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    index = days.get(day, -1)
    if index == -1:
        print(f"Error: Día no reconocido: {day}")
    return index

def analyze_room_schedule(room_data):
    """Analiza el horario de una sala específica."""
    schedule = create_schedule_matrix()
    total_blocks = 45  # 9 bloques x 5 días
    
    # Verificar que tenemos los datos necesarios
    if not isinstance(room_data, dict) or 'Asignaturas' not in room_data:
        print(f"Error: Datos de sala inválidos: {room_data}")
        return None
        
    # Marcar bloques ocupados
    for class_data in room_data.get('Asignaturas', []):
        day_idx = get_day_index(class_data['Dia'])
        block_idx = class_data['Bloque'] - 1
        if 0 <= day_idx < 5 and 0 <= block_idx < 9:
            schedule[day_idx][block_idx] = True
    
    # Contar bloques ocupados y desocupados por día
    daily_stats = []
    for day_schedule in schedule:
        occupied = sum(day_schedule)
        unoccupied = 9 - occupied  # 9 bloques posibles por día
        daily_stats.append({
            'occupied': occupied,
            'unoccupied': unoccupied,
            'compaction_rate': occupied / 9 if occupied > 0 else 0
        })
    
    # Calcular estadísticas totales
    total_occupied = sum(row.count(True) for row in schedule)
    total_unoccupied = total_blocks - total_occupied
    
    return {
        'schedule': schedule,
        'total_occupied': total_occupied,
        'total_unoccupied': total_unoccupied,
        'compaction_rate': total_occupied / total_blocks,
        'daily_stats': daily_stats
    }

def create_summary_table(horarios_salas):
    """Crea una tabla resumen de Ocupacion para todas las salas."""
    summary_data = []
    
    for sala in horarios_salas:
        if not isinstance(sala, dict) or 'Codigo' not in sala:
            print(f"Error: Estructura de sala inválida: {sala}")
            continue
            
        stats = analyze_room_schedule(sala)
        if stats is None:
            continue
            
        summary_data.append({
            'Codigo': sala['Codigo'],
            'Bloques Ocupados': stats['total_occupied'],
            'Bloques Desocupados': stats['total_unoccupied'],
            'Tasa de Ocupacion': round(stats['compaction_rate'] * 100, 2)
        })
    df_summary = pd.DataFrame(summary_data)

    # Añadir fila de totales
    total_row = pd.DataFrame([{
        'Codigo': 'Total',
        'Bloques Ocupados': df_summary['Bloques Ocupados'].sum(),
        'Bloques Desocupados': df_summary['Bloques Desocupados'].sum(),
        'Tasa de Ocupacion': '-'
    }])
    df_summary = pd.concat([df_summary, total_row], ignore_index=True)

    
    # Guardar tabla en CSV
    df_summary.to_csv('resumen_Ocupacion.csv', index=False)
    
    # Mostrar tabla formateada
    print("\nResumen de Ocupacion por Sala:")
    print(df_summary.to_string(index=False))
    
    return df_summary

def create_global_pie_chart(summary_df):
    """Crea un gráfico de torta para la distribución global de bloques."""
    total_occupied = summary_df['Bloques Ocupados'].sum()
    total_unoccupied = summary_df['Bloques Desocupados'].sum()
    
    plt.figure(figsize=(10, 6))
    plt.pie([total_occupied, total_unoccupied],
            labels=['Bloques Ocupados', 'Bloques Desocupados'],
            colors=['#3CB371', '#CD5C5C'],
            autopct='%1.1f%%')
    plt.title('Distribución Global de Ocupación de Bloques')
    plt.savefig('distribucion_bloques.png')
    plt.close()
    
    return total_occupied, total_unoccupied

def create_daily_compaction_chart(horarios_salas):
    """Crea un gráfico de línea para la tasa de Ocupacion diaria."""
    days = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
    daily_total_rates = defaultdict(list)
    
    for sala in horarios_salas:
        stats = analyze_room_schedule(sala)
        for i, day_stat in enumerate(stats['daily_stats']):
            daily_total_rates[days[i]].append(day_stat['compaction_rate'])
    
    # Calcular promedio diario
    daily_averages = [sum(daily_total_rates[day])/len(daily_total_rates[day]) 
                     for day in days]
    
    plt.figure(figsize=(12, 6))
    plt.plot(days, daily_averages, marker='o', linewidth=2, markersize=8)
    plt.title('Tasa de Ocupación Global por Día')
    plt.xlabel('Día de la Semana')
    plt.ylabel('Tasa de Ocupacion Promedio')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('Ocupacion_diaria.png')
    plt.close()
    
    return dict(zip(days, daily_averages))

def save_results(summary_df, total_stats, daily_rates):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': summary_df.to_dict(orient='records'),
            'global_stats': {
                'total_occupied_blocks': int(total_stats[0]),
                'total_unoccupied_blocks': int(total_stats[1]),
                'global_compaction_rate': float(total_stats[0] / (total_stats[0] + total_stats[1]))
            },
            'daily_compaction_rates': daily_rates
        }
        
        with open('metricas_Ocupacion.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    try:
        # Primero leemos el archivo como texto
        with open('../../agent_output/Horarios_salas.json', 'r', encoding='utf-8') as file:
            content = file.read()
        # Luego lo parseamos como JSON
        horarios_salas = json.loads(content)
        if not isinstance(horarios_salas, list):
            print("Error: El archivo no contiene una lista de salas")
            return
    except Exception as e:
        print(f"Error al cargar el archivo: {str(e)}")
        return
    
    # 2. Crear tabla resumen
    summary_df = create_summary_table(horarios_salas)
    if summary_df is None:
        return
    
    # 3. Crear gráfico de torta global
    total_stats = create_global_pie_chart(summary_df)
    if total_stats is None:
        return
    
    # 4. Crear gráfico de línea de Ocupacion diaria
    daily_rates = create_daily_compaction_chart(horarios_salas)
    if daily_rates is None:
        return
    
    # 5. Guardar resultados
    if save_results(summary_df, total_stats, daily_rates):
        print("\nAnálisis de Ocupacion completado exitosamente")
    else:
        print("Error al completar el análisis de Ocupacion")

if __name__ == "__main__":
    main()


Resumen de Ocupacion por Sala:
Codigo  Bloques Ocupados  Bloques Desocupados Tasa de Ocupacion
 KAUS3                12                   33             26.67
 KAUS2                21                   24             46.67
 KAUS1                30                   15             66.67
    K1                19                   26             42.22
    K2                20                   25             44.44
  A106                45                    0             100.0
   CM3                19                   26             42.22
    K3                27                   18              60.0
   CM5                30                   15             66.67
   CM4                30                   15             66.67
    E1                17                   28             37.78
   CM6                16                   29             35.56
   IM3                17                   28             37.78
   IM2                12                   33             26.67
   IM4  